#### 불균형 클래스
#### 클래스 불균형에 대한 이해
#### 클래스 가중치 사용
#### 리셈플링 기법
#### 적절한 평가 지표


In [128]:
# 불균형 데이터 생성(1:9) 악성 0 : 양성 1
from sklearn.datasets import load_breast_cancer
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

data = load_breast_cancer()
X, y = data.data, data.target

# 악성을 소수 클래스로 생성
print(f'악성 양성의 오리지널 비율 : {np.unique(y, return_counts=True)}')

m_index = np.where(y == 0)[0] # 악성 클래스 인덱스 확인
b_index = np.where(y == 1)[0] # 양성 클래스 인덱스 확인

# 악성은 일부만, 양성은 더 많이 사용
# 악성의 19%만 사용, 양성은 전체  1:9
size_30 = int(len(m_index) * 0.19)
selected_m_index = np.random.choice(m_index, size=size_30, replace=False)
selected_b_index = b_index

concatenate_selected_index = np.concatenate([selected_m_index, selected_b_index])
np.random.shuffle(concatenate_selected_index)

X_imb = X[concatenate_selected_index]
y_imb = y[concatenate_selected_index]

# 클래스 분포 확인
unique, counts = np.unique(y_imb, return_counts=True)
print('클래스 분포 : ')
for u, c in zip(unique, counts):
    percentage = c / len(y_imb) * 100
    print(f'클래스 {u} : {c}개, 비율: {percentage:.2f}%')

# 불균형인 상태로 모델 학습
# 스케일링 정규화 StandardScaler
# LogisticRegression 모델 학습
# Pipeline 사용
# 평가는 class report 사용
X_train, X_test, y_train, y_test = train_test_split(X_imb, y_imb, test_size=0.2, random_state=42, stratify=y_imb)
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', LogisticRegression())
])
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred))


악성 양성의 오리지널 비율 : (array([0, 1]), array([212, 357]))
클래스 분포 : 
클래스 0 : 40개, 비율: 10.08%
클래스 1 : 357개, 비율: 89.92%
              precision    recall  f1-score   support

           0       1.00      0.88      0.93         8
           1       0.99      1.00      0.99        72

    accuracy                           0.99        80
   macro avg       0.99      0.94      0.96        80
weighted avg       0.99      0.99      0.99        80



In [131]:
print('불균형 해결 : 클래스 가중치 사용')
# 기존파이프라인의 clf 이름의 객체의 파라메터를 조정
pipe_weighted = pipeline.set_params(clf__class_weight='balanced')
pipe_weighted.fit(X_train, y_train)
y_pred_weighted = pipe_weighted.predict(X_test)
print(classification_report(y_test, y_pred_weighted))

print('가중치 계산')
n_samples = len(y_train)
n_classes = 2
class_weights = np.bincount(y_train)
for i in range(n_classes):
    weight =  n_samples / (n_classes * class_weights[i])
    print(f'클래스 {i}의 가중치 : {weight:.2f}')

불균형 해결 : 클래스 가중치 사용
              precision    recall  f1-score   support

           0       0.78      0.88      0.82         8
           1       0.99      0.97      0.98        72

    accuracy                           0.96        80
   macro avg       0.88      0.92      0.90        80
weighted avg       0.97      0.96      0.96        80

가중치 계산
클래스 0의 가중치 : 4.95
클래스 1의 가중치 : 0.56


In [133]:
from sklearn.ensemble import RandomForestClassifier
print('불균형 : RandomForest(균형모드)')
pipe_rf = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', RandomForestClassifier(class_weight='balanced', random_state=42))
])
pipe_rf.fit(X_train, y_train)
y_pred_rf = pipe_rf.predict(X_test)
print(classification_report(y_test, y_pred_rf))

불균형 : RandomForest(균형모드)
              precision    recall  f1-score   support

           0       1.00      0.62      0.77         8
           1       0.96      1.00      0.98        72

    accuracy                           0.96        80
   macro avg       0.98      0.81      0.87        80
weighted avg       0.96      0.96      0.96        80



In [135]:
# 오버 or 언더 셈플링 기법
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
print('오버샘플링 : SMOTE')
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)
print('리샘플링 후 클래스 분포 : ')
unique, counts = np.unique(y_resampled, return_counts=True)
for u, c in zip(unique, counts):
    percentage = c / len(y_resampled) * 100
    print(f'클래스 {u} : {c}개, 비율: {percentage:.2f}%')
pipe_smote = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', LogisticRegression())
])
pipe_smote.fit(X_resampled, y_resampled)
y_pred_smote = pipe_smote.predict(X_test)
print(classification_report(y_test, y_pred_smote))

print('언더샘플링 : RandomUnderSampler')
rus = RandomUnderSampler(random_state=42)
X_resampled_u, y_resampled_u = rus.fit_resample(X_train, y_train)
print('리샘플링 후 클래스 분포 : ')
unique, counts = np.unique(y_resampled_u, return_counts=True)
for u, c in zip(unique, counts):
    percentage = c / len(y_resampled_u) * 100
    print(f'클래스 {u} : {c}개, 비율: {percentage:.2f}%')
pipe_rus = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', LogisticRegression())
])
pipe_rus.fit(X_resampled_u, y_resampled_u)
y_pred_rus = pipe_rus.predict(X_test)
print(classification_report(y_test, y_pred_rus))

오버샘플링 : SMOTE
리샘플링 후 클래스 분포 : 
클래스 0 : 285개, 비율: 50.00%
클래스 1 : 285개, 비율: 50.00%
              precision    recall  f1-score   support

           0       0.88      0.88      0.88         8
           1       0.99      0.99      0.99        72

    accuracy                           0.97        80
   macro avg       0.93      0.93      0.93        80
weighted avg       0.97      0.97      0.97        80

언더샘플링 : RandomUnderSampler
리샘플링 후 클래스 분포 : 
클래스 0 : 32개, 비율: 50.00%
클래스 1 : 32개, 비율: 50.00%
              precision    recall  f1-score   support

           0       0.64      0.88      0.74         8
           1       0.99      0.94      0.96        72

    accuracy                           0.94        80
   macro avg       0.81      0.91      0.85        80
weighted avg       0.95      0.94      0.94        80

